###  <code>Human Approval (Decision Gate)</code>

In [67]:
from langgraph.graph import StateGraph,START,END
from langgraph.types import Command,interrupt # Command is used to pass user input during human-in-the-loop interactions and interrupt is used to signal pauses in the workflow  
from langchain_google_genai import  ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import  InMemorySaver
from dotenv import load_dotenv
from typing import TypedDict
load_dotenv()

True

In [47]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=2)

In [48]:
class HITLInput(TypedDict):
  title: str
  joke: str

In [49]:
def generate_joke(state: HITLInput):
  prompt = f"Write a short joke about {state['title']}"
  response = model.invoke(prompt)
  return {"joke": response.content}

def review_joke(state: HITLInput):
  decision = interrupt({
    "type": "review",
    "title": state['title'],
    "joke": state['joke'],
    "message": "Please review the joke and say yes / no."
  })
  
  if decision["choice"].lower() == "yes":
    return Command(goto=END)
  else:
    return Command(goto="generate_joke")

In [50]:
graph = StateGraph(HITLInput)

graph.add_node("generate_joke",generate_joke)
graph.add_node("review_joke",review_joke)

graph.add_edge(START,"generate_joke")
graph.add_edge("generate_joke","review_joke")

workflow = graph.compile(checkpointer=InMemorySaver())


In [51]:
initial_state = {"title": "Cats", "joke": ""}
config = {"configurable":{"thread_id":"ID2"}}

In [52]:
result = workflow.invoke(initial_state,config=config)
result

{'title': 'Cats',
 'jock': 'Why was the cat sitting on the computer?\n\nBecause he wanted to keep an eye on the mouse!',
 '__interrupt__': [Interrupt(value={'type': 'review', 'title': 'Cats', 'jock': 'Why was the cat sitting on the computer?\n\nBecause he wanted to keep an eye on the mouse!', 'message': 'Please review the joke and say yes / no.'}, id='ee630320cd3b8a72c27ba16c23cc5b03')]}

In [57]:
result["__interrupt__"][0].value

{'type': 'review',
 'title': 'Cats',
 'jock': 'Why was the cat sitting on the computer?\n\nBecause he wanted to keep an eye on the mouse!',
 'message': 'Please review the joke and say yes / no.'}

In [55]:
user_input = input("Enter 'yes' to accept the joke or 'no' to regenerate: ")
workflow.invoke(Command(resume={"choice": user_input}),config=config)

{'title': 'Cats',
 'jock': "Why are cats such bad poker players?\n\nBecause they're always holding a *paw* in their cards!",
 '__interrupt__': [Interrupt(value={'type': 'review', 'title': 'Cats', 'jock': "Why are cats such bad poker players?\n\nBecause they're always holding a *paw* in their cards!", 'message': 'Please review the joke and say yes / no.'}, id='c2ff8b443bc712afdb286bf576727a33')]}

In [56]:
user_input = input("Enter 'yes' to accept the joke or 'no' to regenerate: ")
workflow.invoke(Command(resume={"choice": user_input}),config=config)

{'title': 'Cats',
 'jock': "Why are cats such bad poker players?\n\nBecause they're always holding a *paw* in their cards!"}

### <code>Human Editing / Correction (Human as Editor)</code>

In [60]:
# State for the editor workflow
class EditorState(TypedDict):
    topic: str
    draft: str
    final_content: str
    revision_count: int

In [61]:
# Node: Generate initial draft
def generate_draft(state: EditorState):
    prompt = f"Write a short paragraph about: {state['topic']}"
    response = model.invoke(prompt)
    return {
        "draft": response.content,
        "revision_count": state.get("revision_count", 0)
    }

# Node: Human editor reviews and can edit the content
def human_editor(state: EditorState):
    # Interrupt to allow human to edit the content
    human_response = interrupt({
        "type": "edit",
        "topic": state["topic"],
        "draft": state["draft"],
        "revision_count": state["revision_count"],
        "message": "Review and edit the draft. Options: 'approve', 'regenerate', or provide edited text"
    })
    
    action = human_response.get("action", "").lower()
    
    if action == "approve":
        # Accept draft as final
        return {"final_content": state["draft"]}
    elif action == "regenerate":
        # Ask AI to regenerate
        return Command(goto="generate_draft")
    else:
        # Human provided edited content - use their version
        edited_content = human_response.get("edited_content", state["draft"])
        return {"final_content": edited_content}

# Node: Finalize the content
def finalize_content(state: EditorState):
    return {"final_content": state.get("final_content", state["draft"])}

In [62]:
# Build the editor workflow graph
editor_graph = StateGraph(EditorState)

# Add nodes
editor_graph.add_node("generate_draft", generate_draft)
editor_graph.add_node("human_editor", human_editor)
editor_graph.add_node("finalize", finalize_content)

# Add edges
editor_graph.add_edge(START, "generate_draft")
editor_graph.add_edge("generate_draft", "human_editor")
editor_graph.add_edge("human_editor", "finalize")
editor_graph.add_edge("finalize", END)

# Compile with checkpointer
editor_workflow = editor_graph.compile(checkpointer=InMemorySaver())

In [63]:
# Start the workflow
editor_state = {"topic": "Benefits of AI in Healthcare", "draft": "", "final_content": "", "revision_count": 0}
editor_config = {"configurable": {"thread_id": "editor-001"}}

editor_result = editor_workflow.invoke(editor_state, config=editor_config)
editor_result

{'topic': 'Benefits of AI in Healthcare',
 'draft': 'Artificial intelligence is revolutionizing healthcare by offering a multitude of benefits. AI-powered tools can significantly enhance diagnostic accuracy, often identifying diseases like cancer or eye conditions with greater precision and speed than human capabilities alone. They also streamline administrative tasks, reducing workload and freeing up medical professionals to focus on patient care. Furthermore, AI can personalize treatment plans by analyzing vast datasets of patient information, predicting treatment efficacy, and suggesting the most optimal therapies. From drug discovery and development to robotic surgery and remote patient monitoring, AI promises to make healthcare more efficient, accessible, and ultimately, more effective.',
 'final_content': '',
 'revision_count': 0,
 '__interrupt__': [Interrupt(value={'type': 'edit', 'topic': 'Benefits of AI in Healthcare', 'draft': 'Artificial intelligence is revolutionizing healt

In [64]:
# View the draft that needs review
interrupt_data = editor_result["__interrupt__"][0].value
print(f"Topic: {interrupt_data['topic']}")
print(f"Revisions: {interrupt_data['revision_count']}")
print(f"\n--- AI Generated Draft ---\n{interrupt_data['draft']}")

Topic: Benefits of AI in Healthcare
Revisions: 0

--- AI Generated Draft ---
Artificial intelligence is revolutionizing healthcare by offering a multitude of benefits. AI-powered tools can significantly enhance diagnostic accuracy, often identifying diseases like cancer or eye conditions with greater precision and speed than human capabilities alone. They also streamline administrative tasks, reducing workload and freeing up medical professionals to focus on patient care. Furthermore, AI can personalize treatment plans by analyzing vast datasets of patient information, predicting treatment efficacy, and suggesting the most optimal therapies. From drug discovery and development to robotic surgery and remote patient monitoring, AI promises to make healthcare more efficient, accessible, and ultimately, more effective.


In [65]:
# Option 1: Approve the draft as-is
# editor_workflow.invoke(Command(resume={"action": "approve"}), config=editor_config)

# Option 2: Ask AI to regenerate
# editor_workflow.invoke(Command(resume={"action": "regenerate"}), config=editor_config)

# Option 3: Provide your own edited version (Human as Editor)
my_edited_content = """
AI is revolutionizing healthcare by enabling faster diagnoses, 
personalized treatments, and improved patient outcomes. 
Machine learning algorithms can analyze medical images with remarkable accuracy,
while predictive models help identify at-risk patients early.
"""

final_result = editor_workflow.invoke(
    Command(resume={"action": "edit", "edited_content": my_edited_content}), 
    config=editor_config
)
print("--- Final Content ---")
print(final_result.get("final_content", "No final content"))

--- Final Content ---

AI is revolutionizing healthcare by enabling faster diagnoses, 
personalized treatments, and improved patient outcomes. 
Machine learning algorithms can analyze medical images with remarkable accuracy,
while predictive models help identify at-risk patients early.

